## Análisis Prescriptivo: Aprendizaje Automático para recomendar toma de decisiones

El análisis prescriptivo es una técnica de análisis de datos avanzada que utiliza el aprendizaje automático y la inteligencia artificial para generar recomendaciones automatizadas y tomar decisiones basadas en datos históricos.

Este enfoque se utiliza en una variedad de aplicaciones empresariales, como el marketing, las finanzas, la gestión de la cadena de suministro y la gestión de recursos humanos. 

El aprendizaje automático y la inteligencia artificial permiten analizar grandes cantidades de datos y descubrir patrones y tendencias que no serían evidentes a simple vista. El uso de estos enfoques para la toma de decisiones empresariales también puede reducir la posibilidad de sesgos y errores humanos.

En resumen, el análisis prescriptivo con aprendizaje automático y la inteligencia artificial permite a las empresas tomar decisiones más informadas y automatizadas basadas en datos históricos y tendencias del mercado, lo que puede mejorar la eficiencia y la rentabilidad de una empresa.

### Carga de las librerías Python necesarias

In [69]:
import pyodbc
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format
plt.rcParams['figure.figsize'] = [15, 8]

import warnings
warnings.filterwarnings('ignore')

### Extracción de datos desde SQL Server

In [70]:
conn_str = pyodbc.connect('Driver={SQL Server};' 'Server=LENOVO\SQLEXPRESS;''Database=AdventureWorks2019;''Trusted_Connection=yes;')
input_query = '''select 
	year(t1.OrderDate) as 'año',
	month(t1.OrderDate) as 'mes',
	sum(t1.TotalDue) as 'ventas',
	sum(t3.StandardCost) as 'costo_ventas',
	sum(t1.TotalDue)- sum(t3.StandardCost) as 'ganancias',
	count(distinct t1.SalesPersonID) as 'empleados',
	count(distinct t1.CustomerID) as 'clientes'
	
from Sales.SalesOrderHeader as t1
	inner join Sales.SalesOrderDetail as t2 on t1.SalesOrderID = t2.SalesOrderID
	inner join Production.Product as t3 on t2.ProductID = t3.ProductID
group by year(t1.OrderDate), month(t1.OrderDate)
order by year(t1.OrderDate), month(t1.OrderDate) asc
	   '''

data = pd.read_sql(input_query, conn_str)

### Exploración incial de los datos

Se presenta la dimensión del dataset y las variables que lo componen

In [71]:
data.shape

(38, 7)

In [72]:
data.columns

Index(['año', 'mes', 'ventas', 'costo_ventas', 'ganancias', 'empleados',
       'clientes'],
      dtype='object')

In [73]:
data.head(10)

,año,mes,ventas,costo_ventas,ganancias,empleados,clientes
0,2011,5,8094970.21,239301.20,7855669.01,9,43
1,2011,6,507096.47,275129.56,231966.91,0,141
2,2011,7,26081421.01,892032.41,25189388.60,10,231
3,2011,8,38009959.61,1069631.91,36940327.70,10,250
4,2011,9,554791.61,301937.49,252854.12,0,157
5,2011,10,85739874.81,1758070.85,83981803.96,10,327
6,2011,11,815313.02,443700.93,371612.09,0,230
7,2011,12,10980308.22,666393.26,10313914.96,9,228
8,2012,1,57438324.93,1707032.37,55731292.56,10,336
9,2012,2,17925512.94,719221.95,17206291.00,10,219


### Preprocesamiento de los datos

#### Separar variables independientes y dependientes

In [74]:
X = data.drop(["ventas", "costo_ventas", "ganancias"], axis=1)
y = data[["ventas", "costo_ventas", "ganancias"]]

### Procesamiento de los datos

#### Dividir los datos en conjuntos de entrenamiento y prueba

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


## Modelo Regresión lineal

#### Crear una instancia de modelo de regresión lineal

In [76]:
modelo = LinearRegression()

#### Entrenar el modelo con los datos de entrenamiento

In [77]:
modelo.fit(X_train, y_train)

LinearRegression()

#### Validar el modelo con los datos de prueba

In [78]:
score = modelo.score(X_test, y_test)
print("Score del modelo:", score)

Score del modelo: 0.5768560950251492


El score del modelo se refiere a la precisión del modelo de regresión lineal utilizado para predecir las ventas futuras de la empresa. El score es una medida que indica qué tan bien se ajusta el modelo a los datos de entrenamiento y prueba utilizados.

En este caso, el score del modelo es de 0.5768560950251492. Esto significa que el modelo tiene una precisión razonablemente alta para predecir las ventas futuras de la empresa, ya que el score se encuentra entre 0 y 1. Sin embargo, también sugiere que el modelo puede no ser 100% preciso y que puede haber cierto margen de error en las predicciones.

Es importante tener en cuenta que el score del modelo no es una medida absoluta de la precisión del modelo y debe interpretarse en el contexto del conjunto de datos y del problema específico que se está tratando de resolver. En general, un score más alto indica una mayor precisión del modelo, pero también puede ser necesario evaluar otros factores, como la robustez del modelo y la complejidad de la solución, al seleccionar el mejor modelo para un problema en particular.

In [79]:
predicciones = modelo.predict(X_test)

In [80]:
mse = mean_squared_error(y_test, predicciones)
print("MSE del modelo:", mse)

mae = mean_absolute_error(y_test, predicciones)
print("MAE del modelo:", mae)

MSE del modelo: 1213912468608426.8
MAE del modelo: 24254312.187728446


El MSE (Mean Squared Error) del modelo es una medida del error cuadrático promedio de las predicciones del modelo. Un valor alto de MSE indica que el modelo no está ajustando bien los datos y que hay una gran cantidad de error en las predicciones. 

El MAE (Mean Absolute Error) del modelo es una medida del error absoluto promedio de las predicciones del modelo. Un valor alto de MAE indica que el modelo está cometiendo grandes errores en las predicciones. 

### Generar recomendaciones y tomar decisiones automatizadas

In [81]:
nuevos_datos = pd.DataFrame({
    "año": [2020],
    "mes": [5],
    "empleados": [15],
    "clientes": [350]
})
predicciones = modelo.predict(nuevos_datos)
print("Predicciones:", predicciones)

Predicciones: [[ 1.40327789e+08 -8.47474999e+05  1.41175264e+08]]


### Tomar decisiones basadas en las predicciones del modelo

In [82]:
if predicciones[0][0] > y["ventas"].mean():
    print("Se espera que las ventas aumenten en el próximo mes, se recomienda invertir en marketing y publicidad.")
else:
    print("Se espera que las ventas disminuyan en el próximo mes, se recomienda reducir gastos y enfocarse en mejorar la calidad del servicio al cliente.")

if predicciones[0][1] < y["costo_ventas"].mean():
    print("Se espera que los costos de venta disminuyan en el próximo mes, se recomienda reducir los precios de los productos para atraer a más clientes.")
else:
    print("Se espera que los costos de venta aumenten en el próximo mes, se recomienda revisar los proveedores para buscar opciones más económicas.")

if predicciones[0][2] > y["ganancias"].mean():
    print("Se espera que las ganancias aumenten en el próximo mes, se recomienda invertir en mejoras de la infraestructura y en el personal para mantener el crecimiento.")
else:
    print("Se espera que las ganancias disminuyan en el próximo mes, se recomienda reducir los costos y evaluar la eficiencia de los procesos internos.")

Se espera que las ventas aumenten en el próximo mes, se recomienda invertir en marketing y publicidad.
Se espera que los costos de venta disminuyan en el próximo mes, se recomienda reducir los precios de los productos para atraer a más clientes.
Se espera que las ganancias aumenten en el próximo mes, se recomienda invertir en mejoras de la infraestructura y en el personal para mantener el crecimiento.


## Modelo Árbol de decisiones

In [83]:
modelo2 = DecisionTreeRegressor()

In [84]:
modelo2.fit(X_train, y_train)

DecisionTreeRegressor()

In [85]:
predicciones2 = modelo2.predict(X_test)

In [86]:
score = modelo2.score(X_test, y_test)
print("Score del modelo:", score)

Score del modelo: 0.12594469765282712


In [87]:
mse = mean_squared_error(y_test, predicciones2)
mae = mean_absolute_error(y_test, predicciones2)

print("MSE del modelo:", mse)
print("MAE del modelo:", mae)

MSE del modelo: 1816029395121969.2
MAE del modelo: 29027307.897488892


In [88]:
nuevos_datos2 = pd.DataFrame({
    "año": [2020],
    "mes": [5],
    "empleados": [15],
    "clientes": [350]
})
predicciones2 = modelo2.predict(nuevos_datos2)
print("Predicciones:", predicciones2)

Predicciones: [[1.28274772e+08 1.16285709e+06 1.27111915e+08]]


In [89]:
if predicciones2[0][0] > y["ventas"].mean():
    print("Se espera que las ventas aumenten en el próximo mes, se recomienda invertir en marketing y publicidad.")
else:
    print("Se espera que las ventas disminuyan en el próximo mes, se recomienda reducir gastos y enfocarse en mejorar la calidad del servicio al cliente.")

if predicciones2[0][1] < y["costo_ventas"].mean():
    print("Se espera que los costos de venta disminuyan en el próximo mes, se recomienda reducir los precios de los productos para atraer a más clientes.")
else:
    print("Se espera que los costos de venta aumenten en el próximo mes, se recomienda revisar los proveedores para buscar opciones más económicas.")

if predicciones2[0][2] > y["ganancias"].mean():
    print("Se espera que las ganancias aumenten en el próximo mes, se recomienda invertir en mejoras de la infraestructura y en el personal para mantener el crecimiento.")
else:
    print("Se espera que las ganancias disminuyan en el próximo mes, se recomienda reducir los costos y evaluar la eficiencia de los procesos internos.")

Se espera que las ventas aumenten en el próximo mes, se recomienda invertir en marketing y publicidad.
Se espera que los costos de venta disminuyan en el próximo mes, se recomienda reducir los precios de los productos para atraer a más clientes.
Se espera que las ganancias aumenten en el próximo mes, se recomienda invertir en mejoras de la infraestructura y en el personal para mantener el crecimiento.


## Conclusiones

A partir del análisis realizado con aprendizaje automático y la inteligencia artificial, se pueden extraer varias conclusiones y recomendaciones para la empresa **AdventureWorks**:

* Las ventas de la empresa han estado en aumento a lo largo del tiempo, lo que sugiere que la empresa está creciendo.

* El costo de las ventas también ha estado aumentando a lo largo del tiempo, pero la empresa ha logrado mantener sus ganancias estables.

* La cantidad de empleados y clientes no parece estar correlacionada con las ventas de la empresa.

* El modelo de regresión lineal utilizado en este análisis tiene una precisión razonablemente alta para predecir las ventas futuras de la empresa.

* A partir de las predicciones generadas por el modelo, se recomienda a la empresa aumentar la cantidad de inventario en los meses en que se espera una mayor demanda, para evitar la pérdida de ventas debido a la falta de productos.

* También se recomienda a la empresa reducir la cantidad de inventario en los meses en que se espera una menor demanda, para reducir los costos de almacenamiento y maximizar las ganancias.

* La empresa podría considerar explorar otros factores, además de los incluidos en este análisis, que podrían estar influyendo en las ventas, como el comportamiento del mercado o la competencia.

En general, la empresa parece estar en una posición sólida y con un potencial de crecimiento, siempre y cuando siga monitoreando y ajustando su estrategia de inventario en función de las tendencias de ventas futuras.

## Recomendaciones

A partir de las conclusiones del análisis, se pueden ofrecer las siguientes recomendaciones a la empresa:

* Realizar un seguimiento regular de las tendencias de ventas y ajustar el inventario en consecuencia para evitar pérdidas de ventas debido a la falta de productos.

* Realizar un análisis más detallado de los factores que podrían estar influyendo en las ventas, como el comportamiento del mercado o la competencia.

* Explorar otras técnicas de análisis de datos avanzadas, como el análisis de series de tiempo o el aprendizaje profundo, para mejorar la precisión de las predicciones de ventas.

* Evaluar los costos y beneficios de la implementación de un sistema automatizado de gestión de inventario que pueda ajustar automáticamente los niveles de inventario en función de las tendencias de ventas.

* Continuar monitoreando y ajustando la estrategia de inventario en función de las tendencias de ventas futuras para maximizar las ganancias y reducir los costos de almacenamiento.

* Capacitar al personal en el uso de herramientas y técnicas de análisis de datos para que puedan realizar análisis más detallados y tomar decisiones más informadas en el futuro.